In [ ]:
pip install pandas psycopg2-binary

In [ ]:
import pandas as pd
import psycopg2 as pg

engine = pg.connect("dbname='clever' user='clever' host='postgres_clever' port='5432' password='clever'")
df = pd.read_sql('select * from public.fmcsa_safer_data', con=engine)

df.head()
